# Comparison in cancer

We compare here the performances of the various factorization methods on multi-omics cancer datasets. The methods are evaluated regarding associations to clinical annotation, survival, and biological annotations (GO, REACTOME, Hallmarks).

## Comparison based on clinical annotations

We define here a function that computes the number of clinical annotations significantly associated with at least one factor, and their selectivity.  The selectivity is defined as the ratio between the number of unique clinical annotations significantly associated with a factor and the number of total (factor,clinical annotation) associations. The clinical metadata need to be stored in a clinical folder containing files named according to each cancer type. 

In [115]:
## Perform clinical annotation-based comparison 
## INPUTS:
# factorizations = already computed factirizations
# clinical = clinical information associated with samples
# col = columns in clinical data on which the analysis will be performed
## OUPUTS: a list containing output values
clinical_comparison <- function(factorizations, clinical, col){

    # Empty containers
    line <- numeric(0)
    line2 <- numeric(0)
    line3 <- numeric(0)

    # For each factorization
    for(i in 1:length(factorizations)){
        
        # Extract sample association
        factors <- factorizations[[i]][[1]]

        # Patient names in factorisation results
        patient.names <- rownames(factors)
        # Patient names in original data
        patient.names.in.file <- as.character(clinical[, 1])
        patient.names.in.file <- toupper(gsub('-', '\\.', patient.names.in.file))
        # Remove non-matching patient names
        is_in_file <- patient.names %in% patient.names.in.file
        if(length(patient.names)!=sum(is_in_file)) {
            factors <- factors[is_in_file, ]
            patient.names <- patient.names[is_in_file]
            rownames(factors)<-patient.names
        }
        # Match indices of patient names
        indices <- match(patient.names, patient.names.in.file)
        # Use indices to extract coresponding survival information
        ordered.clinical.data <- clinical[indices,]        
        
        # Only use column names that are really present in the data
        col_new <- col[col %in% colnames(ordered.clinical.data)]

        col_present<-0
        pvalues<-numeric(0)
        column <- numeric(0)
        clin_erich <- 0 
        
        # Test significance association with clinical annotations
        for(j in col_new){
            
            print(j)
            print(table(ordered.clinical.data[,j]))
            
            if(length(which(table(ordered.clinical.data[,j])>0))>1){
                col_present<-col_present+1
                if(j == "age_at_initial_pathologic_diagnosis" ){
                    column<-c(column,colnames(factors)[which(apply(factors,MARGIN=2, function(x) kruskal.test(x~cut(as.numeric(ordered.clinical.data[,j]),5, include.lowest=TRUE))$p.value)<0.05)])
                    pvalues<-c(pvalues,apply(factors,MARGIN=2, function(x) kruskal.test(x~cut(as.numeric(ordered.clinical.data[,j]),5, include.lowest=TRUE))$p.value))
                    if(min(apply(factors,MARGIN=2, function(x) kruskal.test(x~cut(as.numeric(ordered.clinical.data[,j]),5, include.lowest=TRUE))$p.value))<0.05){
                        clin_erich<-clin_erich+1
                    }
                }else if(j == "days_to_new_tumor_event_after_initial_treatment"){
                    column<-c(column,colnames(factors)[which(apply(factors,MARGIN=2, function(x) kruskal.test(x~cut(as.numeric(ordered.clinical.data[,j]), 3, include.lowest=TRUE))$p.value)<0.05)])
                    pvalues<-c(pvalues,apply(factors,MARGIN=2, function(x) kruskal.test(x~cut(as.numeric(ordered.clinical.data[,j]), 3, include.lowest=TRUE))$p.value))
                    if(min(apply(factors,MARGIN=2, function(x) kruskal.test(x~cut(as.numeric(ordered.clinical.data[,j]), 3, include.lowest=TRUE))$p.value))<0.05){
                        clin_erich<-clin_erich+1
                    }
                }else if(j == "gender" || j == "history_of_neoadjuvant_treatment"){
                    column<-c(column,colnames(factors)[which(apply(factors,MARGIN=2, function(x) wilcox.test(x~ordered.clinical.data[,j])$p.value)<0.05)])
                    pvalues<-c(pvalues,apply(factors,MARGIN=2, function(x) wilcox.test(x~ordered.clinical.data[,j])$p.value))
                    if(min(apply(factors,MARGIN=2, function(x) wilcox.test(x~ordered.clinical.data[,j])$p.value))<0.05){
                        clin_erich<-clin_erich+1
                    }
              }
            }
        }
        signif<-length(which(pvalues<0.05))
        if(clin_erich!=0){
            line<-rbind(line,clin_erich/signif)
        }else{
            line<-rbind(line,0)
        }
        line2<-rbind(line2,length(unique(column)))
        line3<-rbind(line3,clin_erich)

    }
                                 
    out <- list(selectivity=line, nnzero=line2, num.enriched=line3)
    return(out)
}

## Comparison based on survival predictions

We define here the function testing significant associations of factors to survival.

In [110]:
library('survival')

## Perform survival annotation-based comparison 
## INPUTS:
# factorizations = already computed factirizations
# method = methods used for factorization
# survival = survival data associated to the cancer
# out.folder = folder where results will be written
# cancer = name of currently analysed cancer
## OUPUTS: a list containing output values
survival_comparison <- function(factorizations, method, survival, out.folder, cancer){
    
    # Initialize result containers
    factors_cancer <- numeric(0)
    surv_final <- numeric(0)
    
    # For each computed factorisation
    for(i in 1:length(factorizations)){
        
        # Extract sample factors 
        factors <- factorizations[[i]][[1]]
        
        # Patient names in factorisation results
        patient.names <- rownames(factors)
        # Patient names in original data
        patient.names.in.file <- as.character(survival[, 1])
        patient.names.in.file <- toupper(gsub('-', '\\.', patient.names.in.file))
        # Remove non-matching patient names
        is_in_file <- patient.names %in% patient.names.in.file
        if(length(patient.names)!=sum(is_in_file)) {
            factors <- factors[is_in_file, ]
            patient.names <- patient.names[is_in_file]
            rownames(factors)<-patient.names
        }
        # Match indices of patient names
        indices <- match(patient.names, patient.names.in.file)
        # Use indices to extract coresponding survival information
        ordered.survival.data <- survival[indices,]
        # Clean data (assign 0 to NAs)
        ordered.survival.data$Survival[is.na(ordered.survival.data$Survival)] <- 0
        ordered.survival.data$Death[is.na(ordered.survival.data$Death)] <- 0
        # Calculate coxph
        coxph_obj <- coxph(Surv(ordered.survival.data$Survival, ordered.survival.data$Death) ~ factors)
        # P-values (corrected by the number of methods)
        pvalues <- length(factorizations)*as.matrix(coef(summary(coxph_obj))[,5])
        # How many significant? 
        factors_cancer <- c(factors_cancer, sum(pvalues<0.05))
        # Store p-values
        surv_final <- cbind(surv_final, pvalues) 
        
    }
    # Keep -log10 of p-values
    surv_final<-(-log10(surv_final))
    
    # Plot survival pvalues for each cancer type separately
    png(file=paste0(out.folder, "survival_", cancer, ".png"), width = 15, height = 15, units = 'in', res = 200)
    matplot(1:length(method), t(surv_final), 
            col="black", pch=18, xlab="Method", ylab="Pvalues survival", xaxt="none", cex=1.5)
    abline(h = (-log10(0.05)), v=0, col="black", lty=3, lwd=3)
    axis(1, at=1:length(method), labels=colnames(surv_final)) 
    dev.off()

    return(factors_cancer)
}

## Comparison based on association to biological annotations (REACTOME, Hallmarks, GO)

In [104]:
library("fgsea", quietly = TRUE)

## Perform biological annotation-based comparison 
## INPUTS:
# factorizations = already computed factirizations
# path.database = path to a GMT annotation file
# pval.thr = p-value threshold (default to 0.05)
# num.factors = number of factors used during factorisation
## OUPUTS: a list containing output values
bioligcal_comparison <- function(factorizations, path.database, pval.thr=0.05, num.factors){
    pathways <- gmtPathways(path.database)
    
    # Containers for results
    report_number <- numeric(0)
    report_nnzero <- numeric(0)
    report_select <- numeric(0)
    
    # For each factorization method
    for(i in 1:length(factorizations)){
        
        # Metagenes found by factorization method
        metagenes<-factorizations[[i]][[2]][[1]]
        # Rename columns
        colnames(metagenes)<-1:num.factors
        # Rename rows
        rownames(metagenes)<-gsub("\\|",".",rownames(metagenes))
        rownames(metagenes)<-gsub("\\..*","",rownames(metagenes))
        # Remove duplicated gene names that could confuse fgsea
        duplicated_names <- unique(rownames(metagenes)[duplicated(rownames(metagenes))])
        metagenes <- metagenes[!(rownames(metagenes) %in% duplicated_names), ]
        
        # Calculate biological annotation enrichment  
        col2 <- numeric(0)
        path <- numeric(0)
        n <- 0
        for(j in 1:num.factors){
            rnk <- setNames(as.matrix(metagenes[,j]), rownames(metagenes))
            fgseaRes <- fgsea(pathways, rnk, minSize=15, maxSize=500, nperm=1000)
            if(sum(fgseaRes[, padj < pval.thr])!=0){
                n<-n+1
                col2<-rbind(col2,min(fgseaRes[which(fgseaRes[, padj < pval.thr]),]$padj))
                path<-rbind(path,as.matrix(fgseaRes[which(fgseaRes[, padj < pval.thr]),1]))
            }else{
                col2<-rbind(col2,NA)
            }
        }

        if(length(path)==0){
            report_number<-rbind(report_number,0)
        }else{
            report_number<-rbind(report_number,length(unique(path)))
        }
        
        if(length(unique(path))==0){
            report_select<-rbind(report_select,NA)
        }else{
            report_select<-rbind(report_select,length(path)/(length(unique(path))))
        }
        report_nnzero<-rbind(report_nnzero,n)    
    
    }
    
    out <- list(selectivity=report_select, nnzero=report_nnzero, num.enriched=report_number)
    return(out)
}

## Running the comparisons in cancer

The cancer data should be organized into the data folder, each of them having the name of a different cancer type and containin the various omics data (3 omics for our test cases).

In [116]:
# Load the function running the factorization, plus a support function
source("runfactorization.R")
source("log2matrix.R")

# List downloaded cancer data.
# Folder structure should be organized as discussed above.
# Exclude first result as it's the parent folder
cancers <- list.dirs(path = "../data/cancer", full.names = TRUE, recursive = TRUE)[-1]
cancer_names <- list.dirs(path = "../data/cancer", full.names = FALSE, recursive = TRUE)[-1]

# Annotation databases used for biological enrichment
path.database <- "../data/bio_annotations/c2.cp.reactome.v6.2.symbols.gmt" #REACTOME
#path.database <- "../data/bio_annotations/h.all.v6.2.symbols.gmt" #Hallmarks
#path.database <- "../data/bio_annotations/c5.all.v6.2.symbols.gmt" #GO

# Label to identify current run
tag <- format(Sys.time(), "%Y%m%d%H%M%S")
# Folder for comparison results
results_folder <- paste0("../results", tag, "/")
# Create output folder
dir.create(results_folder, showWarnings = FALSE)

# Number of factors used in the paper
num.factors <- 10

# Initialize result containers
clinical_selectivity <- list()
clinical_nnzero <- list()
clinical_num.enriched <- list()
bio_selectivity <- list()
bio_nnzero <- list()
bio_num.enriched <- list()
cancer.list <- list()

# Clinical categories to be used for clinical tests
col <- c("age_at_initial_pathologic_diagnosis",
         "gender",
         "days_to_new_tumor_event_after_initial_treatment",
         "history_of_neoadjuvant_treatment")

In [117]:
# For each cancer dataset
for(i in cancers){

    print(paste0("Now analysing ", i))
    
    # Name of current cancer
    current_cancer <- basename(i)

    # If the expression and miRNA data are not log2-transformed as for those provided by XX et al.
    log2matrix(i,"exp")
    log2matrix(i,"mirna")

    # Perform factorisation
    print("Running factorisation...")
    out <- runfactorization(i, c("log_exp","methy","log_mirna"), num.factors, sep=" ", filtering="sd")
#load("../drive_docs/aml.RData")
    
    # Survival analysis
    print("Running survival analysis...")
    survival <- read.table(paste0(i, "/survival"), sep="\t", header=TRUE)
    out_survival <- survival_comparison(out$factorizations, out$method, survival, 
                                        results_folder, current_cancer)

    # Clinical analysis
    print("Running clinical analysis...")
    clinical <- read.table(paste0("../data/clinical/", current_cancer), sep="\t", header=TRUE)
    out_clinical <- clinical_comparison(out$factorizations, clinical, col)   
    clinical_selectivity[[current_cancer]] <- out_clinical$selectivity
    clinical_nnzero[[current_cancer]] <- out_clinical$nnzero
    clinical_num.enriched[[current_cancer]] <- out_clinical$num.enriched

    # Biological analysis
    print("Running biological analysis...")
    out_bio <- bioligcal_comparison(out$factorizations, path.database, pval.thr=0.05, num.factors)
    bio_selectivity[[current_cancer]] <- out_bio$selectivity
    bio_nnzero[[current_cancer]] <- out_bio$nnzero
    bio_num.enriched[[current_cancer]] <- out_bio$num.enriched
}


[1] "Now analysing ../data/cancer/aml"
[1] "Running factorisation..."
[1] "Running survival analysis..."
[1] "Running clinical analysis..."
[1] "age_at_initial_pathologic_diagnosis"

18 21 22 23 24 25 26 27 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 
 1  2  3  2  1  2  1  1  1  1  3  1  2  2  4  1  1  1  2  1  3  2  2  2  3  1 
47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 
 1  4  1  2  8  1  3  3  4  3  6  4  2  6  7  4  6  5  4  4  5  5  2  2  4  1 
73 74 75 76 77 78 81 82 88 
 3  2  3  8  4  1  3  1  1 
[1] "gender"

FEMALE   MALE 
    79     90 
[1] "history_of_neoadjuvant_treatment"

 No Yes 
124  45 
[1] "age_at_initial_pathologic_diagnosis"

18 21 22 23 24 25 26 27 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 
 1  2  3  2  1  2  1  1  1  1  3  1  2  2  4  1  1  1  2  1  3  2  2  2  3  1 
47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 
 1  4  1  2  8  1  3  3  4  3  6  4  2  6  7  4  6  5  4  4  5 

In [113]:
# Analysed cancer datasets
cancer.list <- names(clinical_selectivity)

# Export results into separated tables
write.table(as.data.frame(clinical_selectivity), paste0(results_folder, "selectivity_clinical_annot.txt"), 
            sep="\t", col.names=cancer.list, row.names=out$method)
write.table(as.data.frame(clinical_nnzero), paste0(results_folder, "nonzero_clinical_annot.txt"),
            sep="\t", col.names=cancer.list, row.names=out$method)
write.table(as.data.frame(clinical_num.enriched), paste0(results_folder, "number_enriched_clinical_annot.txt"),
            sep="\t", col.names=cancer.list, row.names=out$method)
write.table(as.data.frame(bio_selectivity), paste0(results_folder, "selectivity_bio_annot.txt"),
            sep="\t", col.names=cancer.list, row.names=out$method)
write.table(as.data.frame(bio_nnzero), paste0(results_folder, "nonzero_bio_annot.txt"),
            sep="\t", col.names=cancer.list, row.names=out$method)
write.table(as.data.frame(bio_num.enriched), paste0(results_folder, "number_enriched_bio_annot.txt"),
            sep="\t", col.names=cancer.list, row.names=out$method)

## Article figures